# Link Prediction in the Greek Web



First we extract the texts from the folders...

In [102]:
import stemmer as st
def process_word(word):
    """detone and stem word
    """
    new_word = []
    tones = { u'Ά' : u'A' , u'Έ': u'Ε', u'Ί': u'Ι', u'Ϊ': u'Ι',
             u'Ύ': u'Υ', u'Ϋ': u'Υ', u'Ό' : u'Ο', u'Ή': u'Η', u'Ώ': u'Ω'}
    for letter in word:
        try:
            new_word.append(tones[letter])
        except KeyError:
            new_word.append(letter)
    detoned = ''.join(l for l in new_word)
    return st.stem(detoned)
    

In [103]:
import string
def process_text(data):
    with open('dataset/greekstopwords.txt', 'r') as fp:
        stopwords = []
        for line in fp:
            stopwords.append(line.strip().decode('utf-8').upper())
    for domain in data.keys():
        text = data[domain]
        # remove punctuation
        punctuation = set(string.punctuation)    
        doc = ''.join([w for w in text if w not in punctuation])
        # remove stopwords
        doc = [w for w in doc.split() if w not in stopwords]
        doc = [w for w in doc if not re.match(r"$\d+\W+|\b\d+\b|\W+\d+$", w)]
        doc = ' '.join(process_word(w) for w in doc)
        data[domain] = doc
    return data        


In [104]:
import os
import zipfile
import nltk
import pickle

if os.path.isfile('cache/processed_text.pickle'):
    with open('cache/processed_text.pickle', 'rb') as pfile:
        text_data = pickle.load(pfile)
        print "loaded from pickle"
else:
    filenames = os.listdir('dataset/hosts')
    raw_text = {}
    for zipfilename in filenames:
        with zipfile.ZipFile('dataset/hosts/'+zipfilename) as z:
            text = ""
            for filename in z.namelist():
                if not os.path.isdir(filename):
                    with z.open(filename) as f:
                        for line in f:
                            text += line.decode("utf-8").upper()
                            text += " "
            raw_text[zipfilename[:-4]] = text
    text_data = process_text(raw_text)
    with open('cache/processed_text.pickle', 'wb') as pfile:
        pickle.dump(text_data, pfile)

In [105]:
domain_number = {}
for i, domain in enumerate(text_data.keys()):
    domain_number[domain] = i

In [106]:
import re
doc = [word for word in text_data['news247.gr'].split()[:20]]
doc = [w for w in doc if not re.match(r"$\d+\W+|\b\d+\b|\W+\d+$", w)]
for w in doc:
    print w
    #print process_word(word.upper())
    #print st.stem(word.upper())

HTTPNEWS247GREIDISEISPASOK
ΠΕΜΠΤ
ΟΚΤΩΒΡ
ΠΟΛΙΤ
ΚΟΙΝΩΝ
ΟΙΚΟΝΟΜ
ΚΟΣΜ
ΓΝΩΜ
ΠΑΡΑΣΚΗΝΙ
ΠΡΩΤΟΣΕΛΙΔ
ΡΟΗ
ΠΕΡΙΣΣ
ΥΓΕΙ
LIFE
GUIDE
ΤΑΞΙΔ
VIRAL
ΨΥΧΑΓΩΓ
ΦΑΓΗΤ
CELEBRITIES


In [107]:
from __future__ import division
import networkx as nx
import pprint
import random
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split


In [108]:
random.seed(1)
edges = []
with open('dataset/edgelist.txt', 'r')as fp:
    for line in fp:
        edges.append((line.split()[0], line.split()[1]))
print len(edges)
nodes = set([node for _tuple in edges for node in _tuple])
print len(nodes)

2683
2041


In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_text_similarity(data):
    vec = TfidfVectorizer(max_df=0.90, min_df=5, max_features=500,lowercase=False ,
                          analyzer = 'word')
    #vec = TfidfVectorizer(max_df=1, max_features=1000, lowercase=False)
    x = vec.fit_transform(data)
    for word in vec.get_feature_names():
        print word,
    return x
    

In [115]:
X_text = compute_text_similarity(text_data.values())
print X_text.shape
from sklearn.metrics.pairwise import cosine_similarity

cosine = cosine_similarity(X_text)

ABOUT ALL AND ARE AT ATHENS AΔΕΙ AΛΛ AΝΘΡΩΠ AΡΘΡ AΤΟΜ BLOG BY COMMENT COMMENTS CONTACT COOKIES COPYRIGHT DE DESIGN EMAIL FACEBOOK FM FOR FROM GOOGLE GREECE GREEK HOME HOT IN IS IT JULY LED LIFESTYLE LIKE LIVE MAY MEDIA MORE NEW NEWS NEWSLETTER NO OCTOBER OF ON ONLINE OR OUT POST POSTED POWERED RADIO READ RESERVED RIGHTS RSS SEARCH SEPTEMBER SHARE SITE THE THIS TO TOP TWEET TWITTER US VIDEO VIEW WEB WITH YOU YOUR ΑΓ ΑΓΟΡ ΑΓΟΡA ΑΓΩΝ ΑΘΗΝ ΑΘΛΗΤ ΑΘΛΗΤΙΣΜ ΑΚΟΛΟΥΘ ΑΚΟΜ ΑΛΛ ΑΛΛA ΑΝAΠΤΥΞ ΑΝΑΖΗΤΗΣ ΑΝΑΚΟΙΝΩΣ ΑΝΘΡΩΠ ΑΠ ΑΠAΝΤΗΣ ΑΠΟ ΑΠΟΣΤΟΛ ΑΠΟΤΕΛ ΑΠΟΤΕΛΕΣΜ ΑΠΟΦAΣ ΑΠΟΦΑΣ ΑΠΟΨ ΑΠΡΙΛ ΑΠΡΙΛΙ ΑΡΘΡ ΑΡΙΘΜ ΑΡΧ ΑΡΧΕΙ ΑΣΦAΛΕΙ ΑΤΤ ΑΥΓ ΑΥΓΟΥΣΤ ΑΥΤ ΑΥΤA ΑΥΤΟΚΙΝΗΤ ΑΦ ΒAΣ ΒΑΘΜΟΛΟΓ ΒΙΒΛ ΒΙΒΛΙ ΒΙΝΤΕ ΒΟΛ ΒΡ ΒΡΙΣΚ ΓΕΝ ΓΕΡΜΑΝ ΓΕΩΡΓΙ ΓΙAΝΝ ΓΙΑΤ ΓΙΝ ΓΙΩΡΓ ΓΝΩΣΤ ΓΟΝ ΓΡAΦ ΓΡΑΦΕΙ ΓΡΗΓΟΡ ΓΥΝΑΙΚ ΔΕ ΔΕΔΟΜΕΝ ΔΕΚ ΔΕΚΕΜΒΡ ΔΗΛΩΣ ΔΗΜ ΔΗΜΗΤΡ ΔΗΜΙΟΥΡΓ ΔΗΜΟΣ ΔΗΜΟΣΙ ΔΗΜΟΣΙΕΥΘ ΔΗΜΟΣΙΕΥΣ ΔΗΜΟΤ ΔΗΜΟΦΙΛ ΔΙAΒΑΣ ΔΙAΡΚΕΙ ΔΙAΦΟΡ ΔΙΑΒAΣ ΔΙΑΒAΣΤ ΔΙΑΓΩΝΙΣΜ ΔΙΑΤΡΟΦ ΔΙΑΦΗΜΙΣ ΔΙΑΧΕΙΡΙΣ ΔΙΕΘΝ ΔΙΕΥΘΥΝΣ ΔΙΚΑΙ ΔΙΚΑΙΩΜ ΔΙΚΤΥ ΔΙΝ ΔΙΟΙΚΗΣ ΔΡAΣ ΔΡΑΣΤΗΡΙΟΤ

In [116]:
print cosine[domain_number['news247.gr'], domain_number['contra.gr']]

0.478839219864


In [117]:
"""
topic extraction!
http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-topics-extraction-with-nmf-lda-py"""

'\ntopic extraction!\nhttp://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-topics-extraction-with-nmf-lda-py'

In [118]:
def text_similarity(src, dst):
    return cosine[domain_number[src], domain_number[dst]]  
    

In [119]:
non_existent_edges = {}
with open('dataset/not_existing_edges.txt', 'r')as fp:
    for line in fp:
        non_existent_edges[((line.split()[0], line.split()[1]))] = 0 

In [120]:
#X_train, y_train= train_test_split(edges, test_size=0.15, random_state=1)

In [121]:
import networkx as nx
import pprint

"""
#G = nx.read_edgelist('dataset/edgelist.txt', delimiter='\t', create_using=nx.DiGraph())
G = nx.DiGraph()
G.add_edges_from(X_train)
G.add_nodes_from(nodes)"""
G = nx.read_edgelist('dataset/edgelist.txt', delimiter='\t', create_using=nx.DiGraph())
print(len(G.nodes()))
print(len(G.edges()))

2041
2683


In [122]:
non_edges = [edge for edge in list(nx.non_edges(G)) if not non_existent_edges.has_key(edge)]
#4160957

print len(non_edges)

4140197


In [123]:
sum_in_degree = sum(G.in_degree().values())
print sum_in_degree

2683


In [124]:
def neighborhood(src, dst):
    neigh = {}
    neigh['out_src'] = G.out_degree(src)/G.degree(src)
    neigh['in_dst'] = G.in_degree(dst)/G.degree(dst)
    # maybe remove the next two features
    #neigh['in_src'] = G.in_degree(src)/G.degree(src)
    #neigh['out_dst'] = G.out_degree(dst)/G.degree(dst)    
    src_neigh = list(nx.all_neighbors(G, src))
    dst_neigh = list(nx.all_neighbors(G, dst))
    #neigh['common_neighbors'] = len(set(src_neigh).intersection(dst_neigh))/(G.degree(src)+G.degree(dst))
    out_dst = [a for _, a in G.out_edges(dst)]
    in_dst = [a for a, _ in G.in_edges(dst)]
    out_src = [a for _, a in G.out_edges(src)]    
    in_src = [a for a, _ in G.in_edges(src)]
    try:
        neigh['n_common_in'] = len(set(in_src).intersection(in_dst))/(G.in_degree(src)+G.in_degree(dst))
    except ZeroDivisionError:
        neigh['n_common_in'] = 0
    try:
        neigh['n_common_out'] = len(set(out_src).intersection(out_dst))/(G.out_degree(src)+G.out_degree(dst))
    except ZeroDivisionError:
        neigh['n_common_out'] = 0
    return neigh

In [125]:
pagerank = nx.pagerank(G)

In [126]:
betweeness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)
degree = nx.degree_centrality(G)

In [127]:
import math
def adamic_adar(src, dst):
    score = 0
    common = list(set(G.neighbors(src)).intersection(G.neighbors(dst)))
    for node in common:
        score += 1/np.log(len(G.neighbors(node)))
    return sum(1 / math.log(G.degree(w))
                   for w in common)     
    

In [128]:
adamic_adar('news247.gr', 'contra.gr')

3.0020540579927957

In [129]:
def src_features(node):
    features = []
    features.append(pagerank[node])
    features.append(betweeness[node])
    features.append(closeness[node])
    features.append(eigenvector[node])
    features.append(degree[node])
    return features

def dst_features(node):
    features = []
    
    
    

In [130]:
matrix = nx.adjacency_matrix(G)

In [131]:
import igraph as ig
import louvain_igraph as louvain
G_new = ig.Graph()
G_new.add_vertices(G.nodes())
G_new.add_edges(G.edges())

In [132]:
opt = louvain.Optimiser()
partition = opt.find_partition(graph=G_new,partition_class=louvain.SignificanceVertexPartition)

In [133]:
partition = list(partition)
partition_names = []
for com in partition:
    new_com = []
    for node_id in com:
        new_com.append(G_new.vs[node_id]['name'])
    partition_names.append(new_com)
# extract names
partitions = { node : [] for node in G.nodes()}
for com in partition_names:
    for node in com:
        partitions[node].extend(com)

In [134]:
print partitions[G_new.vs[0]['name']]

[u'politikanet.gr', u'katafylli.gr', u'skopelosonline.gr', u'inital.gr', u'arcadiasports.gr', u'bookone1.gr', u'aoristies.gr', u'oraiokastro24.gr', u'siriosfm.gr', u'sdeeth.gr', u'mypatra.gr', u'radioamfilochia.gr', u'ekriti.gr', u'paok-athens.gr', u'sarakis.gr', u'venceremos33.blogspot.gr', u'sportcyclades.gr', u'omospondiadaniolipton.gr', u'pireasnow.gr', u'hysteria.gr', u'radiomanos.gr', u'katanalotis.gr', u'nexusmanagementconsultants.gr', u'misitimi.gr', u'oxafies.com', u'erassitexnikopodosfairo.blogspot.gr', u'pkteam.gr', u'betforthewinners.gr', u'eplski.gr', u'oragiaspor-dramas.gr', u'epskastorias.gr', u'freddonews.gr', u'frontpages.gr']


In [135]:
def partition_check(src, dst):
    counter = 0
    for neigh in G.neighbors(src):
        if neigh in partitions[dst]:
            counter+=1
    """if src in partitions[dst]:
        return 1
    else:
        return 0"""
    return counter
    
partition_check('news247.gr', 'ladylike.gr')


9

In [136]:
def feature_extraction_old(edge):
    src, dst = edge
    f_vector = []
    f_vector.append(text_similarity(src, dst)*2)
    f_vector.append(pagerank[src])
    #f_vector.append(pagerank[dst])
    edge_hood = neighborhood(src, dst)
    # add features: out degree of src, in degree of dst, # of common in with out
    #f_vector.extend([len(edge_hood['out_src']), len(edge_hood['in_dst']), edge_hood['n_common_in'], edge_hood['n_common_out']])
    f_vector.extend(edge_hood.values())    
    return f_vector  
        

In [298]:
def feature_extraction(edge):
    src, dst = edge
    f_vector = []
    f_vector.append(text_similarity(src, dst))
    f_vector.append(pagerank[src])
    f_vector.append(eigenvector[src])
    f_vector.append(eigenvector[dst])
    f_vector.append(pagerank[dst])
    f_vector.append(betweeness[src])
    f_vector.append(betweeness[dst])
    f_vector.append(closeness[dst])
    f_vector.append(closeness[src])
    f_vector.append(adamic_adar(src, dst))
    f_vector.append(partition_check(src, dst))
    return f_vector

"""
def feature_extraction(edge):
    src, dst = edge
    f_vector = []
    f_vector.append(text_similarity(src, dst))
    #f_vector.append(pagerank[src])
    f_vector.append(pagerank[dst])
    f_vector.append(betweeness[src])
    f_vector.append(betweeness[dst])
    f_vector.append(closeness[dst])
    f_vector.append(closeness[src])
    f_vector.append(G.in_degree(dst)/G.degree(dst))
    f_vector.append(len(set(G.neighbors(src)).intersection(G.neighbors(dst))))
    if G.has_edge(dst,src):
        f_vector.append(1)
    else:
        f_vector.append(0)
    f_vector.append(adamic_adar(src, dst))
    f_vector.append(partition_check(src, dst))
    return f_vector
    
    def feature_extraction(edge):
    src, dst = edge
    f_vector = []
    f_vector.append(text_similarity(src, dst))
    #f_vector.append(pagerank[src])
    f_vector.append(eigenvector[dst])
    f_vector.append(pagerank[dst])
    f_vector.append(betweeness[src])
    f_vector.append(betweeness[dst])
    f_vector.append(closeness[dst])
    f_vector.append(closeness[src])
    union = len(set(G.neighbors(src)).union(G.neighbors(dst)))
    intersection = len(set(G.neighbors(src)).intersection(G.neighbors(dst)))
    f_vector.append(intersection)
    if union:
        f_vector.append(intersection/union)
    else:
        f_vector.append(0)
    f_vector.append(adamic_adar(src, dst))
    f_vector.append(partition_check(src, dst))
    if G.has_edge(dst,src):
        f_vector.append(1)
    else:
        f_vector.append(0)
    return f_vector"""

'\ndef feature_extraction(edge):\n    src, dst = edge\n    f_vector = []\n    f_vector.append(text_similarity(src, dst))\n    #f_vector.append(pagerank[src])\n    f_vector.append(pagerank[dst])\n    f_vector.append(betweeness[src])\n    f_vector.append(betweeness[dst])\n    f_vector.append(closeness[dst])\n    f_vector.append(closeness[src])\n    f_vector.append(G.in_degree(dst)/G.degree(dst))\n    f_vector.append(len(set(G.neighbors(src)).intersection(G.neighbors(dst))))\n    if G.has_edge(dst,src):\n        f_vector.append(1)\n    else:\n        f_vector.append(0)\n    f_vector.append(adamic_adar(src, dst))\n    f_vector.append(partition_check(src, dst))\n    return f_vector\n    \n    def feature_extraction(edge):\n    src, dst = edge\n    f_vector = []\n    f_vector.append(text_similarity(src, dst))\n    #f_vector.append(pagerank[src])\n    f_vector.append(eigenvector[dst])\n    f_vector.append(pagerank[dst])\n    f_vector.append(betweeness[src])\n    f_vector.append(betweeness[dst

In [299]:
vector = feature_extraction(('news247.gr','contra.gr'))
print vector

[0.47883921986362765, 0.004009557602747434, 0.2705070374673354, 0.310522072601534, 0.0016805343926659456, 0.00022283606919962717, 2.1252247833905517e-05, 0.005404411764705882, 0.006550802139037433, 3.0020540579927957, 9]


In [300]:
X_train = []
y_train = []
for edge in edges:
    X_train.append(feature_extraction(edge))
    y_train.append(1)
for edge in non_existent_edges:
    X_train.append(feature_extraction(edge))
    y_train.append(0)


In [301]:
X_predict = []
for edge in non_edges:
    X_predict.append(feature_extraction(edge))

In [302]:
X_train_n = np.array(X_train)
y_train_n = np.array(y_train)
X_predict_n = np.array(X_predict)


In [303]:
# scale features
"""from sklearn import preprocessing
X_train_n = preprocessing.scale(X_train_n)
X_predict_n = preprocessing.scale(X_predict_n)"""

'from sklearn import preprocessing\nX_train_n = preprocessing.scale(X_train_n)\nX_predict_n = preprocessing.scale(X_predict_n)'

In [304]:
"""PCA didnt work
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_n = pca.fit_transform(X_train)
X_predict_n = pca.fit_transform(X_predict)"""


'PCA didnt work\nfrom sklearn.decomposition import PCA\npca = PCA(n_components=10)\nX_train_n = pca.fit_transform(X_train)\nX_predict_n = pca.fit_transform(X_predict)'

In [305]:
print X_train_n.shape
print y_train_n.shape
print X_predict_n.shape

(23443, 11)
(23443,)
(4140197, 11)


In [306]:
"""
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
reg = LogisticRegression()
sgd = SGDClassifier(loss="log", penalty="l2")
svm = SVC(probability=True)
#reg.fit(X_train_n, y_train_n)
# pred = reg.predict_proba(X_predict_n)
svm.fit(X_train_n, y_train_n)
pred_svm = svm.predict_proba(X_predict_n)
#sgd.fit(X_train_n, y_train_n)
#pred_sgd = sgd.predict_proba(X_predict_n)
"""



'\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.svm import SVC\nfrom sklearn.linear_model import SGDClassifier\nreg = LogisticRegression()\nsgd = SGDClassifier(loss="log", penalty="l2")\nsvm = SVC(probability=True)\n#reg.fit(X_train_n, y_train_n)\n# pred = reg.predict_proba(X_predict_n)\nsvm.fit(X_train_n, y_train_n)\npred_svm = svm.predict_proba(X_predict_n)\n#sgd.fit(X_train_n, y_train_n)\n#pred_sgd = sgd.predict_proba(X_predict_n)\n'

In [307]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(X_train_n, y_train_n)
pred_xgb = xgb.predict_proba(X_predict_n)

In [308]:
probs = []
print pred_xgb[0]
for t in pred_xgb:
    probs.append(t[1])


[  9.99911010e-01   8.89989897e-05]


In [309]:
indices = sorted(range(len(probs)), key=lambda k: probs[k])[::-1][:453]

In [310]:
print [probs[indices[i]] for i in range(len(indices))]

[0.99908197, 0.99901986, 0.99883097, 0.99878412, 0.99877673, 0.99875224, 0.99863005, 0.99853814, 0.99850428, 0.99844295, 0.99842936, 0.99841297, 0.99839753, 0.99837506, 0.99827886, 0.99827886, 0.99825913, 0.99824393, 0.9982298, 0.99820161, 0.99819034, 0.99816811, 0.99815458, 0.99815458, 0.99815458, 0.99815458, 0.99815458, 0.99815458, 0.99814904, 0.99814701, 0.99810898, 0.9980768, 0.9980768, 0.99807632, 0.99807394, 0.99804282, 0.99798143, 0.99795282, 0.99793303, 0.99792516, 0.99791902, 0.99791795, 0.99791032, 0.99790287, 0.99790287, 0.99789399, 0.99789399, 0.99786925, 0.99786925, 0.9978689, 0.99786359, 0.99786347, 0.99783891, 0.99782324, 0.99779797, 0.99776077, 0.99774987, 0.99774951, 0.99774051, 0.99773139, 0.99773139, 0.99773139, 0.99773139, 0.9977144, 0.99771035, 0.99766123, 0.99765849, 0.99765414, 0.99759895, 0.99759442, 0.99759442, 0.99757665, 0.99756491, 0.99756491, 0.99756491, 0.99756491, 0.99754333, 0.99753821, 0.99753344, 0.99753112, 0.99753016, 0.99752027, 0.99751163, 0.997488

In [311]:
predicted_edges = []
for i in range(len(indices)):
    predicted_edges.append(non_edges[indices[i]])

In [312]:
print predicted_edges[-5:]

[(u'kidsfun.gr', u'proskliseis-kartes.gr'), (u'botanologia.blogspot.gr', u'left.gr'), (u'abtax.gr', u'doe.gr'), (u'efimerides.eu', u'instyle.gr'), (u'efimerides.eu', u'newmoney.gr')]


In [313]:

with open('predicted_edges_xgb.txt', 'w') as fp:
    for site_1, site_2 in predicted_edges:
        fp.write(site_1+'\t'+site_2+'\n')